# Building a herarchial wiki dataset

In [ ]:
# TODO chunck files 

In [168]:
import pandas as pd
import os
import numpy as np
import ete3
import tqdm
import requests

# urlapi

In [173]:
base_url = "https://www.mediawiki.org/w/api.php?"
    #action=help&recursivesubmodules=1
test = "https://www.mediawiki.org/w/api.php?action=help&modules=categorytree&format=json"
test2 = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=categorymembers&cmtitle=Category:Geography&cmlimit=100"
requests.get(test2)

<Response [200]>

# local files

In [9]:
wiki_dir = "/media/kishonylab/ACBA3EA9BA3E6FC6/Alice_datasets/wiki"
cat_path = os.path.join(wiki_dir, 'categorylinks')
page_path = os.path.join(wiki_dir, 'page')

In [154]:
"""
    `page_id` int(8) unsigned NOT NULL AUTO_INCREMENT,
    `page_namespace` int(11) NOT NULL DEFAULT '0',
    `page_title` varbinary(255) NOT NULL DEFAULT '',
    `page_restrictions` tinyblob NOT NULL,
    `page_is_redirect` tinyint(1) unsigned NOT NULL DEFAULT '0',
    `page_is_new` tinyint(1) unsigned NOT NULL DEFAULT '0',
    `page_random` double unsigned NOT NULL DEFAULT '0',
    `page_touched` varbinary(14) NOT NULL DEFAULT '',
    `page_links_updated` varbinary(14) DEFAULT NULL,
    `page_latest` int(8) unsigned NOT NULL DEFAULT '0',
    `page_len` int(8) unsigned NOT NULL DEFAULT '0',
    `page_content_model` varbinary(32) DEFAULT NULL,
    `page_lang` varbinary(35) DEFAULT NULL,
"""
page_cols = ['page_id', 'page_namespace', 'page_title', 'page_restrictions', 'page_is_redirect',
             'page_is_new', 'page_random', 'page_touched', 'page_links_updated', 'page_latest',
             'page_len', 'page_content_model', 'page_lang'] 
page_chucks_size = int(1e4)
page_len = 48258149
def iterate_pages(cols):
    pages = pd.read_csv(page_path, header=None, names=page_cols, chunksize=page_chucks_size, usecols=cols)
    for table in tqdm.tqdm(pages, total=page_len//page_chucks_size):
        yield table

In [164]:
next(iterate_pages(None))




  0%|          | 0/4825 [00:00<?, ?it/s]




,page_id,page_namespace,page_title,page_restrictions,page_is_redirect,page_is_new,page_random,page_touched,page_links_updated,page_latest,page_len,page_content_model,page_lang
0,10,0,AccessibleComputing,NaN,1,0,0.331671,20190728220305,20190105021557,854851586,94,wikitext,NaN
1,12,0,Anarchism,NaN,0,0,0.786172,20190801122349,20190731144500,908640971,102434,wikitext,NaN
2,13,0,AfghanistanHistory,NaN,1,0,0.062150,20190709195906,20190618192734,783865149,90,wikitext,NaN
3,14,0,AfghanistanGeography,NaN,1,0,0.952234,20190627114922,20190618192734,783865160,92,wikitext,NaN
4,15,0,AfghanistanPeople,NaN,1,0,0.574721,20190729015636,20190618192734,783865293,95,wikitext,NaN
5,18,0,AfghanistanCommunications,NaN,1,0,0.751068,20190707171407,20190618192734,783865299,97,wikitext,NaN
6,19,0,AfghanistanTransportations,NaN,1,0,0.674273,20190618191431,20190618192735,783821589,113,wikitext,NaN
7,20,0,AfghanistanMilitary,NaN,1,0,0.118158,20190708195957,20190618192735,783821738,88,wikitext,NaN
8,21,0,AfghanistanTransnationalIssues,NaN,1,0,0.567973,20190730012841,20190618192735,783821743,101,wikitext,NaN
9,23,0,AssistiveTechnology,NaN,1,0,0.723041,20190709010223,20190618192735,783865310,88,wikitext,NaN


In [167]:
# 1) search for the page_id of a selected topic/artical
def find_id(topic, page_namespace=0):
    for cur_page in iterate_pages(['page_id', 'page_title']):
        topic_id = cur_page[cur_page['page_title'] == topic]['page_id']
        if not topic_id.empty:
            return topic_id.values[0]
    return None
find_id('Anarchism')
find_id('Political_culture')




  0%|          | 0/4825 [00:00<?, ?it/s]




12

In [104]:
"""
    `cl_from` int(8) unsigned NOT NULL DEFAULT '0',
    `cl_to` varbinary(255) NOT NULL DEFAULT '',
    `cl_sortkey` varbinary(230) NOT NULL DEFAULT '',
    `cl_timestamp` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    `cl_sortkey_prefix` varbinary(255) NOT NULL DEFAULT '',
    `cl_collation` varbinary(32) NOT NULL DEFAULT '',
    `cl_type` enum('page','subcat','file') NOT NULL DEFAULT 'page',
"""
category_cols = ['cl_from', 'cl_to', 'cl_sortkey', 'cl_timestamp', 'cl_sortkey_prefix', 'cl_collation', 'cl_type'] 
cur_links = next(cat_links)
cat_len = 135264903
link_chucks_size = int(1e5)
def iterate_links():
    cat_links = pd.read_csv(cat_path, header=None, names=category_cols[:3], chunksize=link_chucks_size, sep='\t')
    for table in tqdm.tqdm(cat_links, total=cat_len//link_chucks_size):
        yield table

In [103]:
# 2) Query categorylinks pages with cl_from = topic_id 
anc_filter = lambda x: ~x['cl_to'].str.contains('articles|use|wikipedia|ambiguous', case=False)
def find_cat(topic_id):
    ancestors_list = []
    for cur_links in iterate_links():
        if cur_links['cl_from'].max() < topic_id:
            continue
        if cur_links['cl_from'].min() > topic_id:
            break
        ancestors = cur_links[cur_links['cl_from'] == topic_id]
        ancestors = ancestors[anc_filter(ancestors)]
        ancestors_list.append(ancestors)
    return pd.concat(ancestors_list)
find_cat(topic_id)


  0%|          | 0/1352 [00:00<?, ?it/s]

,cl_from,cl_to,cl_sortkey
6,12,'Anarchism','page'
7,12,'Anti-capitalism','page'
8,12,'Anti-fascism','page'
15,12,'Far-left_politics','page'
17,12,'Libertarian_socialism','page'
18,12,'Political_culture','page'
19,12,'Political_ideologies','page'
20,12,'Social_theories','page'


In [92]:
ancestors

,cl_from,cl_to,cl_sortkey
6,12,'Anarchism','page'
7,12,'Anti-capitalism','page'
8,12,'Anti-fascism','page'
15,12,'Far-left_politics','page'
17,12,'Libertarian_socialism','page'
18,12,'Political_culture','page'
19,12,'Political_ideologies','page'
20,12,'Social_theories','page'


In [85]:
sons

,cl_from,cl_to,cl_sortkey
